In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb, m, p, H, cat
from DGM import DGMPIASolver, DeepPDESolver
from pdes import HBJ, PDE
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG
from FBSDEs import FBSDESolver
from pdes import FBSDE
from sampling import PATH_SPACES
import os
import torch.distributions as D
from torch.quasirandom import SobolEngine

In [4]:
rates = 1
exp = torch.poisson(torch.ones(10, 5))
torch.zeros((5, 1)).uniform_(-torch.rand((5, 1)), torch.rand((5, 1)))

TypeError: uniform_(): argument 'from' (position 1) must be float, not Tensor

In [ ]:
batch_size = 32
var_dim = 100
def sampling(var, domain=(-1, 1)):
    rates = 0.4
    exp = D.exponential.Exponential(rates)
    #means_clusters = torch.clamp(torch.round(exp.sample((var[-1].shape[0], var_dim))), min=0, max=var_dim-1).long()
    means_clusters = torch.randint(1, var_dim, size=(var[-1].shape[0], var_dim))
    bias = ((torch.rand((var[-1].shape[0], 1)))-0.5)*0.4
    means = (torch.rand((var[-1].shape[0], var_dim))-0.5)*1.6 + bias
    samples = []
    for i in range(len(var)):
        m = means.gather(-1, means_clusters[:, None, i])
        b = torch.min(torch.abs(m-domain[0]), torch.abs(m-domain[1]))
        c = torch.rand((var[-1].shape[0], 1))
        samples.append((var[i]-0.5)*torch.min(c, b) + m)
        # need gap between
    return samples

In [ ]:
samples = [torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(var_dim)]
samples = torch.cat(sampling(samples), dim=-1).numpy()

In [ ]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.show()

In [ ]:
samples = torch.cat([(torch.zeros(size=(batch_size, 1)).uniform_()-0.5)*2 for _ in range(var_dim)], dim=-1).numpy()

In [ ]:
fig = make_subplots(rows=1, cols=2)
samples = torch.cat([(torch.zeros(size=(batch_size, 1)).uniform_()-0.5)*2 for _ in range(5)], dim=-1).numpy()
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
samples = torch.cat([(torch.zeros(size=(batch_size, 1)).uniform_()-0.5)*2 for _ in range(50)], dim=-1).numpy()
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=2)

fig.update_layout(
    title="Opinion distribution",
    xaxis_title="$x$",
    xaxis2_title="$x$",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False}, yaxis2={'visible': False, 'showticklabels': False})
fig.show()

In [ ]:
mix = D.Categorical(torch.rand(5,))
comp = D.Independent(D.Normal(
            torch.randn(5, var_dim), torch.rand(5, var_dim)), 1)
gmm = D.MixtureSameFamily(mix, comp)

In [ ]:
def sampm():
    mix = D.Categorical(torch.rand(10,))
    comp = D.Independent(D.Normal(torch.randn(10, var_dim)*0.1 + torch.rand(10, var_dim)-0.5, torch.rand(10, var_dim)), 1)
    gmm = D.MixtureSameFamily(mix, comp)
    return gmm.sample(sample_shape=torch.Size([batch_size]))

In [ ]:
samples = sampm().numpy()

In [ ]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.show()

In [10]:
batch_size = 32
var_dim = 10
def sampling(var, var_dim, mean=0, std=2, domain=(-1, 1)):
    sobol = SobolEngine(var_dim, scramble=True)
    exp = D.Normal(mean, std)
    means_clusters = torch.abs(torch.fmod(torch.round(exp.sample((var[-1].shape[0], var_dim))), var_dim)).long()
    means = (sobol.draw(var[-1].shape[0])-0.5)*2#(torch.rand((var[-1].shape[0], var_dim))-0.5)*2
    samples = []
    for i in range(len(var)):
        m = means.gather(-1, means_clusters[:, None, i])
        b = torch.min(torch.abs(m-domain[0]), torch.abs(m-domain[1]))
        c = torch.rand((var[-1].shape[0], 1))
        samples.append((var[i]-0.5)*2*torch.minimum(b, c) + m)
    return samples

In [14]:
fig = make_subplots(rows=1, cols=2)
samples = torch.cat(sampling([torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(10)], 10, std=4), dim=-1).numpy()
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i*3,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i*3,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
    
samples = torch.cat(sampling([torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(50)], 50, std=1.6), dim=-1).numpy()
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i*3,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i*3,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=2)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="$x$",
    xaxis2_title="$x$",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False}, yaxis2={'visible': False, 'showticklabels': False})
fig.show()

In [5]:
batch_size = 1
var_dim = 1000
samples = [torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(var_dim)]
samples = torch.cat(sampling(samples, 1000, mean=0, std=1), dim=-1).numpy()
fig = go.Figure(data=[go.Histogram(x=samples.flatten(), histnorm='probability')])
fig.show()

In [6]:
a = torch.randn(4, 4)
torch.minimum(a, 1.0)

TypeError: minimum(): argument 'other' (position 2) must be Tensor, not float

In [ ]:
batch_size = 32
var_dim = 50
sobol = SobolEngine(var_dim, scramble=True)
samples = (sobol.draw(batch_size).numpy() -0.5)*2
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.show()